In [3]:
# ============================================
# 04_permutation_and_bootstrap.ipynb
# Permutation test and bootstrap CI for RSA validation
# ============================================

import numpy as np
import pandas as pd
from scipy.stats import spearmanr

# Load previous results
responses = np.load("../data/kay_images.npz")['responses']
roi = np.load("../data/kay_images.npz")['roi']
roi_names = np.load("../data/kay_images.npz")['roi_names']
label_similarity = np.load("../data/semantic_similarity.npy")

n_images = label_similarity.shape[0]
iu = np.triu_indices(n_images, k=1)

def permutation_test(flat_label_sim, flat_brain_corr, n_perm=500, seed=42):
    rng = np.random.default_rng(seed)
    obs_corr, _ = spearmanr(flat_label_sim, flat_brain_corr)
    null_dist = np.zeros(n_perm)
    for i in range(n_perm):
        permuted = rng.permutation(flat_brain_corr)
        null_dist[i], _ = spearmanr(flat_label_sim, permuted)
    p_val = (np.sum(null_dist >= obs_corr) + 1) / (n_perm + 1)
    return obs_corr, p_val

def bootstrap_ci(flat_label_sim, flat_brain_corr, n_boot=500, seed=42):
    rng = np.random.default_rng(seed)
    n = len(flat_label_sim)
    boot_corrs = np.zeros(n_boot)
    for i in range(n_boot):
        idx = rng.choice(np.arange(n), n, replace=True)
        boot_corrs[i], _ = spearmanr(flat_label_sim[idx], flat_brain_corr[idx])
    ci_lower = np.percentile(boot_corrs, 2.5)
    ci_upper = np.percentile(boot_corrs, 97.5)
    return ci_lower, ci_upper

results_perm = []
for roi_idx, roi_name in enumerate(roi_names):
    mask = (roi == roi_idx)
    if np.sum(mask) == 0:
        continue

    fmri_roi = responses[:, mask]
    corr_matrix, _ = spearmanr(fmri_roi, axis=1)
    brain_corr = corr_matrix[:n_images, :n_images]
    flat_label_sim = label_similarity[iu]
    flat_brain_corr = brain_corr[iu]

    rho, p_perm = permutation_test(flat_label_sim, flat_brain_corr)
    ci_low, ci_high = bootstrap_ci(flat_label_sim, flat_brain_corr)
    results_perm.append({"ROI": roi_name, "rho": rho, "p_perm": p_perm,
                         "CI_low": ci_low, "CI_high": ci_high})

df_perm = pd.DataFrame(results_perm)
df_perm.to_csv("../results/rsa_perm_bootstrap.csv", index=False)
print("✅ Saved rsa_perm_bootstrap.csv")

C:\Users\NilGooN\AppData\Local\Temp\ipykernel_3208\41125491.py:21: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  obs_corr, _ = spearmanr(flat_label_sim, flat_brain_corr)
C:\Users\NilGooN\AppData\Local\Temp\ipykernel_3208\41125491.py:25: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  null_dist[i], _ = spearmanr(flat_label_sim, permuted)
C:\Users\NilGooN\AppData\Local\Temp\ipykernel_3208\41125491.py:35: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  boot_corrs[i], _ = spearmanr(flat_label_sim[idx], flat_brain_corr[idx])


✅ Saved rsa_perm_bootstrap.csv
